In [1]:
import numpy as np
import math
import random
import copy
def _NDSetRankLD(data):
	"""
	LD：低维（2维）
	精简第一版算法，直接给每一个等级设定一个最小值
	"""
	# 排序
	index = np.lexsort([data[:, 1], data[:, 0]])
	compareSet = data[index, :]
	# 结果数组
	rankSet = []
	# 维持每个等级的最小值
	minRankValueSet = []
	# 将第一个值加入第一等级序列
	rankSet.append([index[0]])
	minRankValueSet.append(compareSet[0])
	# 记录比较元素前一个元素的等级
	prevValueRank = 0
	for i in range(1, data.shape[0]):
		compareValue = compareSet[i]
		if compareValue[1] < minRankValueSet[prevValueRank][1]: # 如果小于前一个数等级中的最小值
			maxRank = prevValueRank
			minRank = 0
		elif compareValue[1] == minRankValueSet[prevValueRank][1] and compareValue[0] == minRankValueSet[prevValueRank][0]: # 如果等于前一个元素
			minRank = prevValueRank
			maxRank = prevValueRank
		else:
			minRank = prevValueRank + 1
			maxRank = len(rankSet)
		iRank = _findRankLD(rankSet, minRankValueSet, compareValue, minRank, maxRank)
		_addToRankSetLD(rankSet, minRankValueSet, index[i], compareValue, iRank)
		prevValueRank = iRank
	return rankSet

def _findRankLD(rankSet, minRankValueSet, compareValue, minRank, maxRank):
	"""
	寻找nextValue的rank
	:param rankSet:
	:param minRankValueSet:
	:param compareValue:
	:param minRank: 寻找等级范围：最小等级
	:param maxRank: 寻找等级范围：最大等级
	:return: rank
	"""
	while minRank < maxRank:
		midRank = math.floor((maxRank+minRank)/2)  # 二分查找
		midRankMinValue = minRankValueSet[midRank]
		if compareValue[1] < midRankMinValue[1]:
			maxRank = midRank
		elif compareValue[1] > midRankMinValue[1]:
			minRank = midRank + 1
		else:
			return midRank + 1
	return minRank

def _addToRankSetLD(rankSet, minRankValueSet, index, compareValue, iRank):
	# 如果等级超过上限，用处理方式处理
	if iRank >= len(rankSet):
		rankSet.append([index])
		minRankValueSet.append(compareValue)
	else:
		rankSet[iRank].append(index)
		if compareValue[1] < minRankValueSet[iRank][1]:
			minRankValueSet[iRank] = compareValue

def _NDSetRankHD(data):
	"""
	HD：高维（2~n维）
	遍历一遍得出全部等级
	"""
	# 对data排序
	dimension = data.shape[1]
	sortCmp = [data[:, i] for i in reversed(range(dimension))]
	index = np.lexsort(sortCmp)
	compareSet = data[index]
	# 结果数组
	rankSet = []
	# 维持每个等级中与最小值都无关的元素，相当于比较集合
	minRankValueSet = []
	# 第一个值一定是第一等级序列的
	rankSet.append([index[0]])
	minRankValueSet.append([compareSet[0]])
	prevValueRank = 0
	for i in range(1, data.shape[0]):
		prevValue = compareSet[i-1]
		nextValue = compareSet[i]
		relevantValue = _relevantHD(prevValue, nextValue)
		if (prevValue == nextValue).all():  # 如果比较元素和前一个元素相同，则为相同等级
			minRank = prevValueRank
			maxRank = prevValueRank
		elif relevantValue == 0:   # 如果比较元素和前一个元素除第一维值都相同，则必为前一个元素等级加1
			minRank = prevValueRank + 1
			maxRank = prevValueRank + 1
		elif relevantValue == 1:   # 如果前一个元素支配比较元素，则比较元素最小等级为前一元素等级加1
			minRank = prevValueRank + 1
			maxRank = len(rankSet)
		elif relevantValue == 2:   # 如果前一个元素和比较元素互不支配，则不能确定等级
			minRank = 0
			maxRank = len(rankSet)
		elif relevantValue == 3:   # 如果比较元素支配前一个元素（除第一维外的支配），则比较元素最大等级为前一元素等级
			minRank = 0
			maxRank = prevValueRank
		iRank = _findRankHD(rankSet, minRankValueSet, nextValue, minRank, maxRank)
		_addToRankSetHD(rankSet, minRankValueSet, index[i], nextValue, iRank)
		prevValueRank = iRank
	return rankSet

def _addToRankSetHD(rankSet, minRankValueSet, index, compareValue, iRank):
	# 如果等级超过上限，用处理方式处理
	if iRank >= len(rankSet):
		rankSet.append([index])
		minRankValueSet.append([compareValue])
	else:
		rankSet[iRank].append(index)
		minRankValueSet[iRank].append(compareValue)

def _findRankHD(rankSet, minRankValueSet, compareValue, minRank, maxRank):
	while minRank < maxRank:
		midRank = math.floor((maxRank + minRank) / 2)
		midRankMinValueSet = minRankValueSet[midRank]
		isNonDominated = True
		for i in reversed(range(len(midRankMinValueSet))):
			relevantValue = _relevantHD(midRankMinValueSet[i], compareValue)
			if relevantValue < 2:
				minRank = midRank + 1
				isNonDominated = False
				break
			elif relevantValue == 2:
				continue
			elif relevantValue == 3:
				maxRank = midRank
				isNonDominated = False
				break
		if isNonDominated:
			maxRank = midRank
	return minRank

def _relevantHD(prevValue, nextValue):
	"""返回三个值：[第一维不做比较]
	0：值相等
	1：prevValue 支配 nextValue
	2：无关系，互不支配
	3：nextValue 支配 prevValue
	"""
	p = prevValue[1:]
	n = nextValue[1:]
	if (p == n).all():
		return 0
	elif (p <= n).all():
		return 1
	elif (n <= p).all():
		return 3
	else:
		return 2

data = np.array([[ 1,15,14],
             [ 4, 7, 8],
             [ 5, 2,10],
             [ 7, 9, 1],
             [10, 3, 6],
             [10,18,15],
             [14,11, 7],
             [16,17,19],
             [17, 6,18],
             [19,12, 4]])
data = np.array([[ 5,  8, 10],
                 [ 6,  7,  4],
                 [13,  2, 12],
                 [13,  8,  0],
                 [14,  4,  8],
                 [14,  6, 17],
                 [15,  0,  6],
                 [16,  1,  7],
                 [18,  2,  4],
                 [18,  5, 10]])
size = 10
dimension = 3
D = np.empty((size, dimension))
for i in range(size):
    for j in range(dimension):
        D[i][j] = int(random.random() * 20)
D = copy.deepcopy(data)
print(data)

[[ 5  8 10]
 [ 6  7  4]
 [13  2 12]
 [13  8  0]
 [14  4  8]
 [14  6 17]
 [15  0  6]
 [16  1  7]
 [18  2  4]
 [18  5 10]]


In [2]:
d1 = D[:, [0, 1]]
d2 = D[:, [0, 2]]
d3 = D[:, [1, 0]]
d4 = D[:, [1, 2]]
d5 = D[:, [2, 0]]
d6 = D[:, [2, 1]]
rankH = _NDSetRankHD(D)
rankL1 = _NDSetRankLD(d1)
rankL2 = _NDSetRankLD(d2)
rankL3 = _NDSetRankLD(d3)
rankL4 = _NDSetRankLD(d4)
rankL5 = _NDSetRankLD(d5)
rankL6 = _NDSetRankLD(d6)

In [3]:
print(rankH)
print(rankL1)
print(rankL2)
print(rankL3)
print(rankL4)
print(rankL5)
print(rankL6)

[[0, 1, 2, 3, 4, 6, 8], [5, 7], [9]]
[[0, 1, 2, 6], [3, 4, 7], [5, 8], [9]]
[[0, 1, 3], [2, 4, 6, 8], [5, 7], [9]]
[[6, 2, 1, 0], [7, 4, 3], [8, 5], [9]]
[[6, 8, 3], [7, 1], [2, 4], [9], [5, 0]]
[[3, 1, 0], [8, 6, 4, 2], [7, 5], [9]]
[[3, 8, 6], [1, 7], [4, 2], [9], [0, 5]]


In [4]:

# 排序
dimension = data.shape[1]
sortCmp = [data[:, i] for i in reversed(range(dimension))]
index = np.lexsort(sortCmp)
compareSet = np.ones((data.shape[0], dimension+2), int) * -1
compareSet[:, :dimension] = data[index]
compareSet[0, [3, 4]] = [0, 0]
print(compareSet)
# 结果数组
rankSet = []
# 维持每个等级的最小值
minRankValueSet = []
# 将第一个值加入第一等级序列
rankSet.append([index[0]])
minRankValueSet.append(compareSet[0])
# 记录比较元素前一个元素的等级
prevValueRank = 0
for i in range(1, data.shape[0]):
    compareValue = compareSet[i]
    if compareValue[1] < minRankValueSet[prevValueRank][1]: # 如果小于前一个数等级中的最小值
        maxRank = prevValueRank
        minRank = 0
    elif compareValue[1] == minRankValueSet[prevValueRank][1] and compareValue[0] == minRankValueSet[prevValueRank][0]: # 如果等于前一个元素
        minRank = prevValueRank
        maxRank = prevValueRank
    else:
        minRank = prevValueRank + 1
        maxRank = len(rankSet)
    iRank = _findRankLD(rankSet, minRankValueSet, compareValue, minRank, maxRank)
    _addToRankSetLD(rankSet, minRankValueSet, index[i], compareValue, iRank)
    prevValueRank = iRank

[[ 5  8 10  0  0]
 [ 6  7  4 -1 -1]
 [13  2 12 -1 -1]
 [13  8  0 -1 -1]
 [14  4  8 -1 -1]
 [14  6 17 -1 -1]
 [15  0  6 -1 -1]
 [16  1  7 -1 -1]
 [18  2  4 -1 -1]
 [18  5 10 -1 -1]]
